In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss
import plotly
from statistics import median, mean
from selfpkg import readDB, MinToMS
from dataFilter import butter_lowpass_filter, butter_highpass_filter

In [5]:
#function for translate volt to resistance from flapper signal
def transVolt(df):
    for i in range(8):
        df['flapper' + str(i)] = 1000 / df['flapper' + str(i)] - 10

In [32]:
#detect joint by both flapper & acc signal
def jointDetection(df, start=None, end=None, filename='jointDetection'):

    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    #original flapper signal
    #add them all (get rid of bad flapper)
    z = []
    for i in [0, 2, 4, 6]:#range(8):
        if i == 0:
            z.append(df['flapper' + str(i)][startIndex: endIndex])
        else:
            z[0] += df['flapper' + str(i)][startIndex: endIndex]
    
    #lowpass & highpass for flapper signal
    y = []
    for i in range(8):
        if i == 0:
            y.append(butter_highpass_filter(butter_lowpass_filter(df['flapper' + str(i)][startIndex: endIndex], 5, 50, 5), 1, 50, 5))
        else:
            y[0] += butter_highpass_filter(butter_lowpass_filter(df['flapper' + str(i)][startIndex: endIndex], 5, 50, 5), 1, 50, 5)
    
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/60000, 
        y=z[0] - 150,#res,
        name = 'flapper all in one',
        line = dict(
            color = ('rgb(0, 220, 0)'),
            width = 1)
    )
    
    #set threshold height for flapper signal
    thresHeight = 2
    for i in range(len(y[0])):
        if y[0][i] > -thresHeight and y[0][i] < thresHeight:
            y[0][i] = 0
    
    y[0] = abs(y[0])
    #generate peaks for signal and time    
    peaksIndex, _ = ss.find_peaks(y[0])
    peaksIndexPlus = [x + startIndex for x in peaksIndex]
    
    i = 0
    j = 1
    thresInterval = 50 #num of points of peaks distance
    res = []
    #for every potential joints, we may have several peaks for it. So we try to merge clost peaks, only remain one peak for one joint    
    while(j < len(peaksIndexPlus)):
        if  j == len(peaksIndexPlus) - 1:
            res.append(int((peaksIndexPlus[i] + peaksIndexPlus[j]) / 2))
#             cur = y[0][peaksIndex[i]-startIndex: peaksIndex[j]-startIndex]
#             if cur.size != 0:
#                 res.append(np.argmax(cur, axis=0) + startIndex)
            i = j
            j += 1       
        else:
            if peaksIndexPlus[j] - peaksIndexPlus[j-1] > thresInterval:
                j -= 1
#                 cur = y[0][peaksIndex[i]-startIndex: peaksIndex[j]-startIndex]
#                 if cur.size != 0:
                res.append(int((peaksIndexPlus[i] + peaksIndexPlus[j]) / 2))
#                     res.append(np.argmax(cur, axis=0) + startIndex)
                j += 1
                i = j
                j += 1       
            else:
                j += 1
    
    res = [x-13 for x in res] #fix shift from convolution
    timeConversionVal = 60000
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=y[0],#res,
        name = 'flapper all in one',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    
    trace4 = go.Scattergl(
        x = (df.time[res])/timeConversionVal,
        y = [0] * len(res),
        mode = 'markers',
        name = 'peaks',
        marker = dict(
            color = ('rgb(255, 0, 0)'))
    )
    ############################### Flapper above & Acc following ########################################
    #set threshold for three direction acc
    accX = []
    accY = []
    accZ = []
    acc = []
    thresAccHeight = 15
    for i in range(startIndex, endIndex+1):
        if df.accX[i] > -thresAccHeight and df.accX[i] < thresAccHeight:
            accX.append(0)
        else:
            accX.append(df.accX[i])
         
        if df.accY[i] > -thresAccHeight and df.accY[i] < thresAccHeight:
            accY.append(0)
        else:
            accY.append(df.accY[i])
        
        if df.accZ[i] > -thresAccHeight and df.accZ[i] < thresAccHeight:
            accZ.append(0)
        else:
            accZ.append(df.accZ[i])
    #add three direction acc to one singal direction, set threshold height again
    for i in range(len(accX)):
        cur = (accX[i] ** 2 + accY[i] ** 2 + accZ[i] ** 2) ** (1/2)
        if cur < 21:
            acc.append(0)
        else:
            acc.append(cur)
    #generate peaks both for acceleration & time
    peaksAccIndex, _ = ss.find_peaks(acc)
    peaksAccIndexPlus = [x + startIndex for x in peaksAccIndex]
    #same as flapper, we need to merge close peaks
    i = 0
    j = 1
    resAcc = []
    while(j < len(peaksAccIndexPlus)):
        if  j == len(peaksAccIndexPlus) - 1:
            resAcc.append(int((peaksAccIndexPlus[i] + peaksAccIndexPlus[j]) / 2))
            #res.append((y[0].index(max(y[0][peaksIndex[i-startIndex: j-startIndex]]))) + startIndex)
            i = j
            j += 1       
        else:
            if peaksAccIndexPlus[j] - peaksAccIndexPlus[j-1] > thresInterval:
                j -= 1
                resAcc.append(int((peaksAccIndexPlus[i] + peaksAccIndexPlus[j]) / 2))
                #res.append((y[0].index(max(y[0][peaksIndex[i-startIndex: j-startIndex]]))) + startIndex)
                j += 1
                i = j
                j += 1       
            else:
                j += 1
    
    traceAcc = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=[x-100 for x in acc],
        name = 'accX',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    
    traceAccPoint = go.Scattergl(
        x=df.time[resAcc]/timeConversionVal,
        y=[-10] * len(resAcc),
        mode = 'markers',
        name = 'peaksAcc',
        marker = dict(
            color = ('rgb(127, 127, 127)'))
    )
    
    data = [trace0, trace4, trace5, traceAcc, traceAccPoint]
    
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Friction')
                      )

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html')
    
    return res, resAcc #peaks index list

In [33]:
#generate speed chart & add tow col (v, p) at the end of dataframe
def speedChart(df, resFlapper, resAcc, start, end, filename='speedChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    #calculate velocity for every peak
#     time1 = velocityCal(df.time, resFlapper)
    time2 = velocityCal(df.time, resAcc)
    
    #calculate mean of velocity
    errorPercent = 0.1
    mode = 9200
#     avg1 = mean([x for x in time1 if x >= mode * (1 - errorPercent) and x <= mode * (1 + errorPercent)])
    avg2 = mean([x for x in time2 if x >= mode * (1 - errorPercent) and x <= mode * (1 + errorPercent)])
    
    con1 = []
    con2 = []
    #if one point lost, double time between two peaks; treble for two and so on...
    #So we shift the point with two/three/four times of mean
#     for i in time1.index:
#         diff = abs(time1[i] - avg1)
#         con1.append(1 - diff/avg1)
    for i in time2.index:
#         if time2[i]/avg2 >= 1.8 and time2[i]/avg2 <= 2.2:
#             time2[i] = time2[i] / 2
#         if time2[i]/avg2 >= 2.8 and time2[i]/avg2 <= 3.2:
#             time2[i] = time2[i] / 3
#         if time2[i]/avg2 >= 3.8 and time2[i]/avg2 <= 4.2:
#             time2[i] = time2[i] / 4
        diff = abs(time2[i] - avg2)
        con2.append(1 - diff/avg2)
    
    #fill the result to df
    df['Velocity'] = 0
    df['Probability'] = 0
    df['Velocity'][resAcc] = time2
    df['Probability'][resAcc] = con2
    
    first = []
    shrinkVal = 0.1 #other points between two joint, lower the possibility
    thresPro = 0.85 #threshold for selecting a joint
    
    #fill out empty velocity & probability in df
    #connect tow close confident points(probability > 0.85), calculate points between them by linear function
    #first & last segment, only use one point
    for index, row in df[startIndex: endIndex].iterrows():
        if len(first) == 0:
            if row['Probability'] < thresPro:
                continue
            else:
                first = [row['Velocity'], row['Probability'], row['time'], index]
                for i in range(startIndex, index):
                    df['Velocity'][i] = first[0]
                    df['Probability'][i] = first[1] - shrinkVal
        else:
            if index != endIndex -1:
                if row['Probability'] < thresPro:
                    continue
                else:
                    k = (row['Velocity'] - first[0]) / (row['time'] - first[2])
                    b = row['Velocity'] - k * row['time']
                    for i in range(first[3]+1, index):
                        df['Velocity'][i] = k * df['time'][i] + b
                        df['Probability'][i] = (1 - abs(df['Velocity'][i]-avg2) / avg2) - shrinkVal
                    first = [row['Velocity'], row['Probability'], row['time'], index]
            else:
                for i in range(first[3], endIndex):
                    df['Velocity'][i] = first[0]
                    df['Probability'][i] = first[1] - shrinkVal
                    
    timeConversionVal = 60000
#     traceAvg1 = go.Scattergl(
#         x = df.time/timeConversionVal,
#         y = [avg1] * len(df.index),
#         mode = 'lines',
#         name = 'flapperAvg',
#         line = dict(
#             color = ('rgb(255, 0, 0)')
#         )
#     )
        
#     traceFlapper = go.Scattergl(
#         x = df.time[resFlapper]/timeConversionVal,
#         y = time1,
#         mode = 'markers',
#         name = 'flapper',
#         marker = dict(
#             color = ('rgb(255, 0, 0)')
#         )
#     )
    
    traceAvg2 = go.Scattergl(
        x = df.time/timeConversionVal,
        y = [avg2] * len(df.index),
        mode = 'lines',
        name = 'accelerationAvg',
        line = dict(
            color = ('rgb(0, 0, 255)')
        )
    )
    
    traceAcceleration = go.Scattergl(
        x = df.time[resAcc]/timeConversionVal,
        y = time2,
        mode = 'markers',
        name = 'acceleration',
        marker = dict(
            color = ('rgb(0, 0, 255)')
        )
    )
    
    traceFit = go.Scattergl(
        x = df.time[startIndex: endIndex] / timeConversionVal,
        y = df.Velocity[startIndex: endIndex],
        mode = 'lines',
        name = 'velocity',
        line = dict(
            color = ('rgb(127, 127, 127)')
        )
    )
    
    data = [traceAcceleration, traceAvg2, traceFit]
    
    layout = dict(title = filename,
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'timeInterval')
                  )
    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') 
    
    return con1, con2

In [34]:
#function for calculate the time interval between peaks
def velocityCal(df, peaksIndex):
    dfTime = df.iloc[peaksIndex] #get sub dataframe
    prev = dfTime[0: 1] #cache the first 
    for i, _ in dfTime.iteritems():
        cur = dfTime[i] #cache cur
        dfTime[i] = dfTime[i] - prev #get difference
        prev = cur #set prev from cur (cur val we've cached before)
    return dfTime

In [35]:
csv_database = 'sqlite:///sa_run01.db'  ##for sa
df = readDB(csv_database)
transVolt(df)
jointFlapper, jointAcc= jointDetection(df, 9.3, 18.2)
p1, p2 = speedChart(df, jointFlapper, jointAcc, 13, 175)

In [401]:
csv_database = 'sqlite:///Cambridge2.db'  ##for cambridge
df2 = readDB(csv_database)
transVolt(df2)
jointFlapper, jointAcc= jointDetection(df, 9, 17)
# p1, p2 = speedChart(df, jointFlapper, jointAcc, 13, 175)

In [414]:
csv_database = 'sqlite:///SouthHarrison.db'  #Indiana
df = readDB(csv_database)
transVolt(df)
jointFlapper, jointAcc= jointDetection(df, 13, 175)
p1, p2 = speedChart(df, jointFlapper, jointAcc, 13, 175)

In [452]:
def getTimeJoint(df):
    timeJoint = []
    for index, row in df.iterrows():
        if row['Probability'] >= 0.9:
            timeJoint.append(row['time'])
    return timeJoint

In [415]:
###
listMaxMag = []
listMaxIndex = []
for i in range(8):
    if i == 0 or i == 1 or i == 3:
        listMaxMag.append([i])
        listMaxIndex.append([i])
        continue
    sublist4Mag = []
    sublist4Index = []
    for time in timeJoint:
        index = df.index[df['time'] == time].tolist()[0]
        windowSize = 60 #ofpoints
        startIndex = int(index - windowSize/2)
        endIndex = int(index + windowSize/2)
        sublist4Mag.append(df['flapper' + str(i)][startIndex: endIndex].max(axis=0))
        sublist4Index.append(df['flapper' + str(i)][startIndex: endIndex].idxmax(axis=0))
    listMaxMag.append(sublist4Mag)
    listMaxIndex.append(sublist4Index)

sizeMag = 0.5
fig = plotly.tools.make_subplots(rows=5, cols=1)
count = 1
for i in [2, 4, 5, 6, 7]:
    trace0 = go.Histogram(
        x=listMaxMag[i],
        name='Flapper' + str(i) + '+',
        xbins=dict(
            start=0,
            end=100,
            size=sizeMag
        ),
        marker=dict(
            color='#FFD7E9',
        ),
        opacity=1
    )
    fig.append_trace(trace0, count, 1)
    fig['layout']['xaxis' + str(count)].update(title = 'magnitude')
    fig['layout']['yaxis' + str(count)].update(title = 'count')
    count += 1
plotly.offline.plot(fig, filename = 'test4MagHistogram.html') #offline generate

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]



'file:///Users/meng/Documents/dataPreprocessing/venv/source/test4MagHistogram.html'

In [321]:
listWidth = []
for i in [0, 1, 2, 3, 4, 5, 6, 7]:
    if i == 0 or i == 1 or i == 3:
        listWidth.append([i])
        continue
    res = ss.peak_widths(df['flapper' + str(i)], listMaxIndex[i], rel_height = 1)
    listWidth.append(res[0].tolist())

sizeMag = 20
fig = plotly.tools.make_subplots(rows=5, cols=1)
count = 1
for i in [2, 4, 5, 6, 7]:
    trace0 = go.Histogram(
        x=listWidth[i],
        name='Flapper' + str(i) + '+',
        xbins=dict(
            start=0,
            end=2000,
            size=sizeMag
        ),
        marker=dict(
            color='#FFD7E9',
        ),
        opacity=1
    )
    fig.append_trace(trace0, count, 1)
    fig['layout']['xaxis' + str(count)].update(title = 'timeWidth(ms)')
    fig['layout']['yaxis' + str(count)].update(title = 'count')
    count += 1
    
plotly.offline.plot(fig, filename = 'test4WidthHistogram.html') #offline generate

/Users/meng/Documents/dataPreprocessing/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: PeakPropertyWarning:

some peaks have a prominence of 0

/Users/meng/Documents/dataPreprocessing/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: PeakPropertyWarning:

some peaks have a width of 0



This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]



'file:///Users/meng/Documents/dataPreprocessing/venv/source/test4WidthHistogram.html'

In [326]:
medianMag = []
medianWidth = []
for i in range(8):
    medianMag.append(median(sorted(listMaxMag[i])))
    medianWidth.append(median(sorted(listWidth[i])))

[0, 1, 0.8469656535172767, 3, 3.28413813292228, 2.2534446531925383, 2.4908852245820015, 3.9610408528630714]
[0, 1, 139.7792411414266, 3, 118.27526881720405, 121.8352758352994, 154.65317276515998, 136.9803921568673]


In [339]:
correlist = []
for i in range(8):
    sublist = []
    if i == 0 or i == 1 or i == 3:
        correlist.append([i])
        continue
    k = - (medianMag[i] / (medianWidth[i] - 20))
    b = - (medianWidth[i] * k)
    sublist.append(0)
    for t in range(1, int(medianWidth[i] / 20) + 1):
        y = k * (t * 20) + b
        sublist.append(y)
    sublist.append(0)
    correlist.append(sublist)

In [340]:
print(correlist)

[[0], [1], [0, 0.8469656535172767, 0.7055445449101114, 0.5641234363029461, 0.42270232769578087, 0.2812812190886156, 0.1398601104814503, 0], [3], [0, 3.2841381329222794, 2.615783180053942, 1.947428227185605, 1.2790732743172675, 0.6107183214489305, 0], [0, 2.2534446531925383, 1.8108780406491425, 1.368311428105747, 0.9257448155623513, 0.48317820301895553, 0.04061159047556018, 0], [0, 2.490885224582002, 2.1209147035121125, 1.7509441824422236, 1.3809736613723347, 1.0110031403024455, 0.6410326192325564, 0.27106209816266746, 0], [0, 3.9610408528630714, 3.2838263590782617, 2.606611865293452, 1.9293973715086423, 1.2521828777238326, 0.5749683839390229, 0]]


In [374]:
conv = []
for i in range(8):
    if i == 0 or i == 1 or i == 3:
        conv.append([i])
        continue
    sublist  = []
    for time in timeJoint:
        index = df.index[df['time'] == time].tolist()[0]
        windowSize = 60 #ofpoints
        startIndex = int(index - windowSize/2)
        endIndex = int(index + windowSize/2)
        l1 = df['flapper' + str(i)][startIndex: endIndex].tolist()
        l2 = correlist[i]
        l1 = [(x - mean(l1)) / (np.std(l1) * len(l1)) for x in l1]
        l2 = [(x - mean(l2)) / (np.std(l2)) for x in l2]
#         sublist.append(max(np.correlate(l1, l2)))
        sublist.append(max(np.correlate(l1, l2, mode = 'full')))
    conv.append(sublist)

In [378]:
sizeMag = 0.1
fig = plotly.tools.make_subplots(rows=5, cols=1)
count = 1
for i in [2, 4, 5, 6, 7]:
    trace0 = go.Histogram(
        x=conv[i],
        name='Flapper' + str(i) + '+',
        xbins=dict(
            start=-0.25,
            end=0.25,
#             size=sizeMag
        ),
        marker=dict(
            color='#FFD7E9',
        ),
        opacity=1
    )
    fig.append_trace(trace0, count, 1)
    fig['layout']['xaxis' + str(count)].update(range=[-0.25, 0.25])
#     fig['layout']['yaxis' + str(count)].update(title = 'count')
    count += 1
    
plotly.offline.plot(fig, filename = 'test4convHistogram.html') #offline generate

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]



'file:///Users/meng/Documents/dataPreprocessing/venv/source/test4convHistogram.html'

In [466]:
from scipy.signal import savgol_filter
from statistics import stdev
def FFTsingle(df, dfFlapper, start=None, end=None):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    fs = 50
    cutoff = 1
    order = 5

    freq = np.fft.fftfreq(len(df[startIndex: endIndex].index), d=0.02)
    sp = np.fft.fft(butter_highpass_filter(dfFlapper[startIndex: endIndex], cutoff, fs, order))

    trimFreq = []
    trimSp = []
    for i in range(len(freq)):
        if freq[i] >= 0 and freq[i] <= 100:
            trimFreq.append(freq[i])
            trimSp.append(sp[i].real)
    #fitting
    if len(trimSp)  > 5:
        trimSpFilter = savgol_filter(trimSp, 5, 3)
    else:
        trimSpFilter = trimSp
    
    list = []
    for i in range(len(trimFreq)):
        diff = trimSp[i] - trimSpFilter[i]
        list.append(diff)
    
    sd = stdev(list)
    
    trace0 = go.Scattergl(
        x=trimFreq,#freq,
        y=trimSp,#sp.real,
        name = 'real',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1),
    )
    
    trace1 = go.Scattergl(
        x=trimFreq,#freq,
        y=trimSpFilter,
        name = 'fitting line',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1),
    )
    
    data = [trace0, trace1]    
    return data, sd

In [503]:
def FFTFrontAllinOne(df, start=None, end=None, filename='FFTFrontAllinOne'):
    
    fig = plotly.tools.make_subplots(rows=4, cols=1, subplot_titles=('flapper0', 'flapper1', 'flapper2', 'flapper3'))
    list = []
    for i in [0, 1, 2, 3]:
        dfFlapper = df['flapper' + str(i)]
        data, sd = FFTsingle(df, dfFlapper, start, end)
        list.append(sd)
        for trace in data:
            fig.append_trace(trace, i+1, 1)
            fig['layout']['xaxis' + str(i+1)].update(title='Frequency(Hz)')
            fig['layout']['yaxis' + str(i+1)].update(title='Amplitude')
            
    fig['layout'].update(title='FFT for front flapper')
    plotly.offline.plot(fig, filename = filename + '.html')
    return list

In [504]:
def FFTBackAllinOne(df, start=None, end=None, filename='FFTBackAllinOne'):
    
    fig = plotly.tools.make_subplots(rows=4, cols=1, subplot_titles=('flapper4', 'flapper5', 'flapper6', 'flapper7'))
    list = []
    for i in [4, 5, 6, 7]:
        dfFlapper = df['flapper' + str(i)]
        data, sd = FFTsingle(df, dfFlapper, start, end)
        list.append(sd)
        for trace in data:
            fig.append_trace(trace, i-3, 1)
            fig['layout']['xaxis' + str(i-3)].update(title='Frequency(Hz)')
            fig['layout']['yaxis' + str(i-3)].update(title='Amplitude')
    
    fig['layout'].update(title='FFT for back flapper')
    plotly.offline.plot(fig, filename = filename + '.html')
    return list

In [422]:
mid = timeJoint[303] / 60000
# mid = 35.5
start = mid - 0.01
end = mid + 0.01

list2 = FFTBackAllinOne(df, start, end, 'FFTBackLeak')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [441]:
def FFTpointSelectSingle(df, dfFlapper, start=None, end=None):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    fs = 50
    cutoff = 1
    order = 5

    freq = np.fft.fftfreq(len(df[startIndex: endIndex].index), d=0.02)
    sp = np.fft.fft(butter_highpass_filter(dfFlapper[startIndex: endIndex], cutoff, fs, order))

    trimFreq = []
    trimSp = []
    for i in range(len(freq)):
        if freq[i] >= 0 and freq[i] <= 100:
            trimFreq.append(freq[i])
            trimSp.append(sp[i].real)
    
#     trimFreq = [round(x, 3) for x in trimFreq]
#     trimSp = [round(x, 3) for x in trimSp]
    
#     listofMag = [x for x in trimFreq if x > 8 and x < 8.5]
    listofMag = [x for x in trimFreq if x > 2.75 and x < 3.25]
    
#     index1 = trimFreq.index(2.083)
#     index2 = trimFreq.index(3.125)
#     index3 = 10
    
    listMag = []
    
#     listMag.append(trimSp[index1])
#     listMag.append(trimSp[index2])
#     listMag.append(trimSp[index3])
    if len(listofMag) != 0:
        index = trimFreq.index(listofMag[0])
        res = trimSp[index]
    else:
        res = 0
    return res

In [442]:
sizeMag = 10
fig = plotly.tools.make_subplots(rows=4, cols=1)

for i in [4, 5, 6, 7]:
    dfFlapper = df['flapper' + str(i)]
    listMag = []
    for time in timeJoint:
        time = time/60000
        start = time - 0.01
        end = time + 0.01
        mag = FFTpointSelectSingle(df, dfFlapper, start, end)
        listMag.append(mag)
    trace0 = go.Histogram(
        x=listMag,
        name='Flapper' + str(i) + '+',
        marker=dict(
            color='#FFD7E9',
        ),
        opacity=1
    )
    fig.append_trace(trace0, i-3, 1)
    
plotly.offline.plot(fig, filename = 'test4Histogram(3.125Hz).html') #offline generate

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



'file:///Users/meng/Documents/dataPreprocessing/venv/source/test4Histogram(3.125Hz).html'

In [449]:
def fftStd(df, timeJoint):
    listRes1 = [[], [], [], []]
    listRes2 = [[], [], [], []]
    for time in timeJoint:
        time = time/60000
        start = time - 0.01
        end = time + 0.01
        list1 = FFTFrontAllinOne(df, start, end, 'FFTFrontAllJoint')
        list2 = FFTBackAllinOne(df, start, end, 'FFTBackAllJoint')
        for i in range(len(list1)):
            listRes1[i].append(list1[i])
        for i in range(len(list2)):
            listRes2[i].append(list2[i])
    return listRes1, listRes2

In [501]:
def abnormalityonFFT(listFront, listBack, timeJoint):
    listFrontRes = []
    listBackRes = []
    threshold = 0.99
    for i in range(len(listFront)):
        sort = sorted(listFront[i])
        stdMedian = median(sort)
        start = int(threshold * (len(sort)-1))
        end = int(len(sort))
        for k in range(start, end):
            if sort[k] > 1.1 * stdMedian:
                listFrontRes.append(timeJoint[listFront[i].index(sort[k])])
    
    for i in range(len(listBack)):
        sort = sorted(listBack[i])
        stdMedian = median(sort)
        start = int(threshold * (len(sort)-1))
        end = int(len(sort))
        for k in range(start, end):
            if sort[k] > 1.1 * stdMedian:
                listBackRes.append(timeJoint[listBack[i].index(sort[k])])
    
    return listFrontRes, listBackRes                   

In [469]:
def fftStdHistogram(listRes1, listRes2):
    sizeMag = 5
    fig1 = plotly.tools.make_subplots(rows=4, cols=1)
    for i in range(len(listRes1)):
        trace1 = go.Histogram(
            x=listRes1[i],
            name='Flapper' + str(i),
            marker=dict(
                color='#FFD7E9',
            ),
            opacity=1
        )
        fig1.append_trace(trace1, i+1, 1)
        fig1['layout']['xaxis' + str(i+1)].update(title='std')
        fig1['layout']['yaxis' + str(i+1)].update(title = 'count')

    plotly.offline.plot(fig1, filename = 'test4fftstdHistogramFront.html')
    
    fig2 = plotly.tools.make_subplots(rows=4, cols=1)
    for i in range(len(listRes2)):
        trace2 = go.Histogram(
            x=listRes2[i],
            name='Flapper' + str(i+4),
            marker=dict(
                color='#FFD7E9',
            ),
            opacity=1
        )
        fig2.append_trace(trace2, i+1, 1)
        fig2['layout']['xaxis' + str(i+1)].update(title='std')
        fig2['layout']['yaxis' + str(i+1)].update(title = 'count')

    plotly.offline.plot(fig2, filename = 'test4fftstdHistogramBack.html')

In [467]:
csv_database = 'sqlite:///SouthHarrison.db'  #Indiana
df = readDB(csv_database)
transVolt(df)
jointFlapper, jointAcc= jointDetection(df, 13, 175)
p1, p2 = speedChart(df, jointFlapper, jointAcc, 13, 175)
timeJoint = getTimeJoint(df)
listFront, listBack = fftStd(df, timeJoint)
fftStdHistogram(listFront, listBack)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

...

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [470]:
fftStdHistogram(listFront, listBack)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [492]:
front, back = abnormalityonFFT(listFront, listBack, timeJoint)

In [499]:
print(front)

[4843263.0, 1725448.0, 3253522.0, 3416815.0, 4396985.0, 3344549.0, 3638419.0, 4604893.0, 3174408.0, 2147552.0, 3510180.0, 2858503.0, 2907483.0, 3515172.0, 3278930.0, 974869.0, 3314081.0, 5567199.0, 6214311.0, 9774985.0, 5916439.0, 6302801.0, 2813875.0, 9282821.0, 7208781.0, 5872596.0, 8613265.0, 3334252.0, 3298831.0, 8860350.0, 5853193.0, 3515172.0, 896182.0, 1208242.0, 1330279.0, 5270942.0, 852217.0, 1335192.0, 1892791.0, 886683.0, 5567199.0, 1057272.0, 1912561.0, 2897727.0, 2152629.0, 4064379.0, 905972.0, 1716065.0]


In [500]:
print(back)

[2917343.0, 1744998.0, 1730225.0, 1740163.0, 1647438.0, 856799.0, 2162894.0, 2966509.0, 2124509.0, 2873364.0, 921069.0, 1120645.0, 1608568.0, 2152629.0, 3243338.0, 2147552.0, 5567199.0, 2853813.0, 2577872.0, 4604893.0, 1495922.0, 2476584.0, 2471743.0, 2773416.0, 2921982.0, 896182.0, 911092.0, 1588976.0, 2768727.0, 1608568.0, 2167600.0, 2863320.0, 866800.0, 916083.0, 1042678.0, 856799.0, 2813875.0, 3113402.0, 2853813.0, 2921982.0, 2966509.0, 2152629.0, 2384659.0, 2167600.0, 1037880.0, 2917343.0, 2863320.0, 2873364.0, 7424972.0, 2873364.0, 5406557.0, 8588462.0, 7414276.0, 6736588.0, 5270942.0, 5716530.0, 8183707.0, 6785887.0, 6855564.0, 7813287.0, 8860350.0, 6705823.0, 3711488.0, 5382001.0]


In [511]:
mid = back[11]/60000
start = mid - 0.1
end = mid + 0.1
FFTFrontAllinOne(df, start, end)
FFTBackAllinOne(df, start, end)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



[27.573233122087395, 6.75997478345026, 8.449887296914834, 9.162585265338999]

In [512]:
print(mid)

18.677416666666666
